In [1]:
import open3d as o3d
import numpy as np
import os
import mcubes
import h5py

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
os.getcwd()

'/home/piai/RIM-Net'

In [3]:
pcd = o3d.io.read_point_cloud("20_desk_gaussian.ply")
points = np.asarray(pcd.points)

In [4]:
o3d.visualization.draw_geometries([pcd])

In [5]:
print("Original bounds:")
print("min:", points.min(axis=0))
print("max:", points.max(axis=0))

# center
center = points.mean(axis=0)
points = points - center

# scale to fit [-0.5, 0.5]
scale = np.max(np.linalg.norm(points, axis=1))
points = points / (2.0 * scale)

print("After normalization:")
print("min:", points.min(axis=0))
print("max:", points.max(axis=0))

Original bounds:
min: [-0.50112176 -0.24990103 -0.22437435]
max: [0.50135267 0.25072199 0.2244482 ]
After normalization:
min: [-0.37010126 -0.18396379 -0.29673422]
max: [0.36913654 0.1852022  0.03423342]


In [6]:
pcd.points = o3d.utility.Vector3dVector(points)
o3d.io.write_point_cloud("table_normalized.ply", pcd)

print("Saved table_normalized.ply")

Saved table_normalized.ply


In [7]:
RES = 64

pcd = o3d.io.read_point_cloud("table_normalized.ply")
points = np.asarray(pcd.points)

# map [-0.5,0.5] → [0,63]
idx = ((points + 0.5) * RES).astype(np.int32)
idx = np.clip(idx, 0, RES - 1)

vox = np.zeros((RES, RES, RES), dtype=np.float32)
vox[idx[:, 0], idx[:, 1], idx[:, 2]] = 1.0

vox = vox[np.newaxis, ..., np.newaxis]  # [1,64,64,64,1]

np.save("table_vox.npy", vox)

print("Voxel grid shape:", vox.shape)
print("Occupied voxels:", int(vox.sum()))

Voxel grid shape: (1, 64, 64, 64, 1)
Occupied voxels: 4498


In [8]:
vox = np.load("table_vox.npy")[0, :, :, :, 0]
vertices, triangles = mcubes.marching_cubes(vox, 0.5)

print("Vertices:", len(vertices))
print("Triangles:", len(triangles))

Vertices: 3630
Triangles: 7256


In [9]:
vox = np.load("table_vox.npy")

points = np.zeros((1, 32*32*32, 3), dtype=np.float32)
values = np.zeros((1, 32*32*32, 1), dtype=np.float32)

with h5py.File("table_test.hdf5", "w") as f:
    f.create_dataset("voxels", data=vox)
    f.create_dataset("points_64", data=points)
    f.create_dataset("values_64", data=values)

print("Saved table_test.hdf5")

Saved table_test.hdf5


## model test

In [10]:
import tensorflow as tf
from RIM_model import RIM

In [11]:
tf.compat.v1.reset_default_graph()

class DummyConfig:
    checkpoint_dir = "checkpoint"
    result_dir = "results"

sess = tf.compat.v1.Session()

rim = RIM(
    sess=sess,
    real_size=64,
    points_per_shape=32*32*32,
    is_training=False,
    z_dim=128,
    ef_dim=32,
    gf_dim=256,
    dataset_name="table",
    checkpoint_dir="checkpoint",
    result_dir="results",
    data_dir=".",
    branch_num=8)

print("✅ RIM model initialized with branch_num=8")

2025-12-17 11:45:19.362398: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2025-12-17 11:45:19.371859: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2600000000 Hz
2025-12-17 11:45:19.373319: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x67b86b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-12-17 11:45:19.373391: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



conv3d in (1, 64, 64, 64, 1) out (1, 32, 32, 32, 32)
conv3d in (1, 32, 32, 32, 32) out (1, 16, 16, 16, 64)
conv3d in (1, 16, 16, 16, 64) out (1, 8, 8, 8, 128)
conv3d in (1, 8, 8, 8, 128) out (1, 4, 4, 4, 256)
conv3d in (1, 4, 4, 4, 256) out (1, 1, 1, 1, 128)
pointz (?, 131)
linear in [None, 128] out (None, 256)
linear in [None, 256] out (None, 256)
linear in [None, 128] out (None, 256)
linear in [None, 256] out (None, 256)
linear in [None, 128] out (None, 256)
linear in [None, 256] out (None, 256)
linear in [None, 131] out (None, 1024)
linear in [None, 1024] out (None, 256)
linear in [None, 256] out (None, 14)
linear in [None, 131] out (None, 1024)
linear in [None, 1024] out (None, 256)
linear in [None, 256] out (None, 14)
linear in [None, 131] out (None, 1024)
linear in [None, 1024] out (None, 256)
linear in [None, 256] out (None, 14)
linear in [None, 131] out (None, 1024)
linear in [None, 1024] out (None, 256)
linear in [None, 256] out (None, 14)
linear in [None, 131] out (None, 1024

Instructions for updating:
Use the `axis` argument instead


the network is done
conv3d in (1, 64, 64, 64, 1) out (1, 32, 32, 32, 32)
conv3d in (1, 32, 32, 32, 32) out (1, 16, 16, 16, 64)
conv3d in (1, 16, 16, 16, 64) out (1, 8, 8, 8, 128)
conv3d in (1, 8, 8, 8, 128) out (1, 4, 4, 4, 256)
conv3d in (1, 4, 4, 4, 256) out (1, 1, 1, 1, 128)
pointz (?, 131)
linear in [None, 128] out (None, 256)
linear in [None, 256] out (None, 256)
linear in [None, 128] out (None, 256)
linear in [None, 256] out (None, 256)
linear in [None, 128] out (None, 256)
linear in [None, 256] out (None, 256)
linear in [None, 131] out (None, 1024)
linear in [None, 1024] out (None, 256)
linear in [None, 256] out (None, 14)
linear in [None, 131] out (None, 1024)
linear in [None, 1024] out (None, 256)
linear in [None, 256] out (None, 14)
linear in [None, 131] out (None, 1024)
linear in [None, 1024] out (None, 256)
linear in [None, 256] out (None, 14)
linear in [None, 131] out (None, 1024)
linear in [None, 1024] out (None, 256)
linear in [None, 256] out (None, 14)
linear in [None, 

✅ RIM model initialized with branch_num=8


In [12]:
os.makedirs("results", exist_ok=True)
print("results/ directory ready")

results/ directory ready


In [13]:
rim.test_dae(DummyConfig())

 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from checkpoint/table/RIM.model-0


INFO:tensorflow:Restoring parameters from checkpoint/table/RIM.model-0


 [*] Success to read RIM.model-0
 [*] Load SUCCESS
[RIM] Saving voxel parts as .npy...
[RIM]  Level 1, parts = 2
   saved: results/t0_vox_level1_part0.npy shape: (66, 66, 66) max: 0.9989979
   saved: results/t0_vox_level1_part1.npy shape: (66, 66, 66) max: 0.99460036
[RIM]  Level 2, parts = 4
   saved: results/t0_vox_level2_part0.npy shape: (66, 66, 66) max: 0.94060636
   saved: results/t0_vox_level2_part1.npy shape: (66, 66, 66) max: 0.999989
   saved: results/t0_vox_level2_part2.npy shape: (66, 66, 66) max: 0.073260196
   saved: results/t0_vox_level2_part3.npy shape: (66, 66, 66) max: 0.99979144
[RIM]  Level 3, parts = 8
   saved: results/t0_vox_level3_part0.npy shape: (66, 66, 66) max: 0.94458365
   saved: results/t0_vox_level3_part1.npy shape: (66, 66, 66) max: 0.9873654
   saved: results/t0_vox_level3_part2.npy shape: (66, 66, 66) max: 0.99981296
   saved: results/t0_vox_level3_part3.npy shape: (66, 66, 66) max: 0.9988924
   saved: results/t0_vox_level3_part4.npy shape: (66, 66, 6

In [14]:
opt2 = o3d.io.read_point_cloud("/home/piai/RIM-Net/results/0_avox_2.ply")

o3d.visualization.draw_geometries([opt2])

In [15]:
input = o3d.io.read_point_cloud("/home/piai/RIM-Net/results/0_input.ply")

o3d.visualization.draw_geometries([input])

In [23]:
part1 = o3d.io.read_point_cloud("/home/piai/RIM-Net/results/0_vox_2_0.ply")

o3d.visualization.draw_geometries([part1])

## parts

In [18]:
parts = []
for i in range(4):
    parts.append(np.load(f"results/t0_vox_level2_part{i}.npy"))

parts = np.stack(parts, axis=-1)   # (66,66,66,4)

# assign each voxel to strongest part
part_id_grid = np.argmax(parts, axis=-1)  # (66,66,66)

print("Unique part IDs:", np.unique(part_id_grid))

Unique part IDs: [0 1 2 3]


In [19]:
grid = np.linspace(-0.5, 0.5, parts.shape[0])
X, Y, Z = np.meshgrid(grid, grid, grid, indexing="ij")

for pid in range(4):
    mask = part_id_grid == pid
    if mask.sum() < 50:
        print(f"Part {pid}: too small, skip")
        continue

    z_range = Z[mask].ptp()
    xy_range = np.sqrt(
        X[mask].ptp()**2 + Y[mask].ptp()**2
    )
    z_mean = Z[mask].mean()

    print(
        f"Part {pid}: z_range={z_range:.3f}, "
        f"xy_range={xy_range:.3f}, z_mean={z_mean:.3f}"
    )

Part 0: z_range=1.000, xy_range=1.414, z_mean=-0.000
Part 1: z_range=0.415, xy_range=1.023, z_mean=-0.002
Part 2: z_range=0.308, xy_range=0.972, z_mean=0.059
Part 3: z_range=0.308, xy_range=0.972, z_mean=0.005


In [ ]:
Legs → large z_range, small xy_range, low z_mean

Tabletop → small z_range, large xy_range, high z_mean

In [34]:
pcd = o3d.io.read_point_cloud("table_normalized.ply")
points = np.asarray(pcd.points)

idx = ((points + 0.5) * 66).astype(int)
idx = np.clip(idx, 0, 65)

point_part_ids = part_id_grid[
    idx[:,0], idx[:,1], idx[:,2]]

In [41]:
leg_mask = point_part_ids == 3
leg_points = points[leg_mask]

legs_pcd = o3d.geometry.PointCloud()
legs_pcd.points = o3d.utility.Vector3dVector(leg_points)

o3d.io.write_point_cloud("table_legs.ply", legs_pcd)

True

In [42]:
legs = o3d.io.read_point_cloud("table_legs.ply")

o3d.visualization.draw_geometries([legs])